In [1]:
import numpy as np
import h5py
from utils.DRP_uitls import *
import pyvista as pv

In [2]:
#Download binary image from DRP
file_url = "https://www.digitalrocksportal.org/projects/175/images/159992/download/"
download_file(file_url,'./data/3D_binary_MicrostructureData.mat')

#Download velocity data from DRP
file_url = "https://www.digitalrocksportal.org/projects/175/images/159989/download/"
download_file(file_url,'./data/VX.mat')
file_url = "https://www.digitalrocksportal.org/projects/175/images/159990/download/"
download_file(file_url,'./data/VY.mat')
file_url = "https://www.digitalrocksportal.org/projects/175/images/159991/download/"
download_file(file_url,'./data/VZ.mat')

In [6]:
#Load binay image data
arrays = {}
f = h5py.File('./data/3D_binary_MicrostructureData.mat','r')
for k, v in f.items():
    arrays[k] = np.array(v)
f.close()

img=arrays['Microstruct']
scale=arrays['scale'].flatten()[0]
print('Image Info',img.shape,
      f'scale={scale} mm/voxel\n',
      f'dimension={np.array(img.shape)*scale*1e3} mm',)

Image Info (565, 525, 1071) scale=1.6812865497076024e-05 mm/voxel
 dimension=[ 9.49926901  8.82675439 18.00657895] mm


In [10]:
#Load velocity data
Velocity = {}
for vfile in ['./data/VX.mat','./data/VY.mat','./data/VZ.mat']:
    f = h5py.File(vfile,'r')
    for k, v in f.items():
        Velocity[k] = np.array(v)*1e6 #um/s
    f.close()
Velocity.pop('scale')

Vel_magn=np.sqrt(np.power(Velocity['VX'],2)+np.power(Velocity['VY'],2)+np.power(Velocity['VZ'],2))

C:\Users\pc\AppData\Local\Temp/ipykernel_33456/4179418003.py:4: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(vfile)


In [11]:
#Convert numpy array to paraview vti image
NX,NY,NZ=1000,500,500 #Correct axis order (input data is reversed)
NX,NY,NZ=100,200,200 #Correct axis order (input data is reversed)
os.makedirs("./data", exist_ok=True)

vtkimg = pv.UniformGrid(np.array([NX,NY,NZ])+1)
vtkimg['MetaImage']=np.einsum('ijk->kji', img[0:NZ,0:NY,0:NX]).flatten(order="F")
vtkimg.save("./data/image_binary.vti")

In [12]:
Vel_field=np.zeros([np.prod([NX,NY,NZ]),3],dtype=np.float32)
for i,v in enumerate(['VX','VY','VZ']):
    Vel_field[:,i]=np.einsum('ijk->kji', Velocity[v][0:NZ,0:NY,0:NX]).flatten(order="F")

vtkimg = pv.UniformGrid(np.array([NX,NY,NZ])+1)
vtkimg.cell_arrays['Velocity(um/s)']=Vel_field
vtkimg.save("./data/image_vel.vti")